In [1]:
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 6.2 MB/s eta 0:00:00


In [ ]:
from qiskit import QuantumCircuit, transpile, assemble, Aer, execute
from qiskit.visualization import plot_histogram
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.circuit.library import QFT
from qiskit.extensions import UnitaryGate
from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer
import numpy as np
import time
import random

circ =  0
from qiskit.circuit.library import QFT
from qiskit.extensions import UnitaryGate

def qft(n):
    qc = QuantumCircuit(n)
    qc.append(QFT(n, do_swaps=False, inverse=True), range(n))
    return qc

def c_amod15(a, power):
    #if a not in [2, 7, 8, 11, 13]:
     #   raise ValueError("'a' must be 2, 7, 8, 11, or 13")
    U = QuantumCircuit(4)
    for iteration in range(power):
        if a in [2, 13]:
            U.swap(0, 1)
            U.swap(1, 2)
            U.swap(2, 3)
        if a in [7, 8]:
            U.swap(2, 3)
            U.swap(1, 2)
            U.swap(0, 1)
        if a == 11:
            U.swap(1, 3)
            U.swap(0, 2)
        if a in [7, 11, 13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = "%i^%i mod" % (a, power)
    c_U = U.control()
    return c_U

def shor_algorithm(N, a):
    #Initialize qubits
    n = 8 #Number of qubits
    n_ancilla = 4 #Number of ancilla qubits
    qc = QuantumCircuit(n + n_ancilla, n)

    #Hadamard gates
    qc.h(range(n))

    #modular exponentiation
    for q in range(n):
        qc.append(c_amod15(a, 2**q),
                  [q] + list(range(n, n + n_ancilla)))

    #Inverse Fourier transform
    qc.append(qft(n).inverse(), range(n))

    #Measure the first n qubits
    qc.measure(range(n), range(n))

    #Run the circuit on a quantum simulator
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend, shots=2000)
    result = job.result()

    #classical post-processing
    counts = result.get_counts()
    fractions = {}
    for output in counts:
        binary = output[::-1]
        fraction = int(binary, 2) / (2 ** n)
        fractions[binary] = fractions.get(binary, 0) + counts[output] / 1024

    #Find the period r
    probabilities = list(fractions.values())
    measured_states = list(fractions.keys())
    phase_estimate = measured_states[np.argmax(probabilities)]
    r = int(phase_estimate, 2)
    #print(r)

    #Find the factors using the period
    if r % 2 != 0:
        print("Failed to find factors. Try again with a different 'a'.")
    else:
        x = int((a ** (r / 2)) % N)
        if (x + 1) % N == 0:
            print("Failed to find factors. Try again with a different 'a'.")
        else:
            factors = np.gcd(x + 1, N), np.gcd(x - 1, N)
            return factors
    #qc.draw(output='mpl')

In [ ]:
def classical_trial_division(n):
    factors = []
    for i in range(2, int(n**0.5) + 1):
        while n % i == 0:
            factors.append(i)
            n = n / i
    if n > 1:
        factors.append(int(n))
    return factors

In [ ]:
N = 35
a = 11 #random.randint(2, N-1)
start_time = time.time()
factors = shor_algorithm(N, a)
shor_time = time.time() - start_time

start_time = time.time()
classical_factors = classical_trial_division(N)
classical_time = time.time() - start_time

print("Factors using Shor's algorithm:", factors)
print("Time taken by Shor's algorithm:", shor_time, "seconds")
print("Factors using classical trial division:", classical_factors)
print("Time taken by classical trial division:", classical_time, "seconds")

Factors using Shor's algorithm: (7, 5)
Time taken by Shor's algorithm: 1.7623755931854248 seconds
Factors using classical trial division: [5, 7]
Time taken by classical trial division: 7.033348083496094e-05 seconds
